<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/3-introduction-to-keras-and-tensorflow/01_tensorflow_fundametal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##First steps with TensorFlow

As we know, training a neural network revolves around the following
concepts:

- First, low-level tensor manipulation—the infrastructure that underlies all modern machine learning. This translates to TensorFlow APIs:
  - Tensors, including special tensors that store the network’s state (variables)
  - Tensor operations such as addition, relu, matmul
  - Backpropagation, a way to compute the gradient of mathematical expressions

- Second, high-level deep learning concepts. This translates to Keras APIs:
  - Layers, which are combined into a model
  - A loss function, which defines the feedback signal used for learning
  - An optimizer, which determines how learning proceeds
  - Metrics to evaluate model performance, such as accuracy
  - A training loop that performs mini-batch stochastic gradient descent

Now let’s take a deeper dive into how all of these different concepts can be
approached in practice using TensorFlow and Keras.

##Setup

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

##Constant tensors and variables

To do anything in TensorFlow, we’re going to need some tensors. Tensors need to be created with some initial value. 

For instance, you could create all-ones or all-zeros tensors, or tensors of values drawn from a random distribution.

**All-ones or all-zeros tensors**

In [2]:
x = tf.ones(shape=(2, 1))
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)


In [3]:
x = np.ones(shape=(2, 1))
print(x)

[[1.]
 [1.]]


In [4]:
x = tf.zeros(shape=(2, 1))
print(x)

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


In [5]:
x = np.zeros(shape=(2, 1))
print(x)

[[0.]
 [0.]]


**Random tensors**

In [6]:
# Tensor of random values drawn from a normal distribution with mean 0 and standard deviation 1.
x = tf.random.normal(shape=(3, 1), mean=0.0, stddev=1.0)
print(x)

tf.Tensor(
[[-0.65337884]
 [-1.0298071 ]
 [-1.17121   ]], shape=(3, 1), dtype=float32)


In [7]:
x = np.random.normal(size=(3, 1), loc=0.0, scale=1.0)
print(x)

[[-0.34242538]
 [-0.84452594]
 [ 0.04355254]]


In [8]:
# Tensor of random values drawn from a uniform distribution between 0 and 1.
x = tf.random.uniform(shape=(3, 1), minval=0.0, maxval=1.0)
print(x)

tf.Tensor(
[[0.15898514]
 [0.57103324]
 [0.6071217 ]], shape=(3, 1), dtype=float32)


In [9]:
x = np.random.uniform(size=(3, 1), low=0.0, high=1.0)
print(x)

[[0.27621971]
 [0.21226116]
 [0.57525445]]


A significant difference between NumPy arrays and TensorFlow tensors is that Tensor-
Flow tensors aren’t assignable: they’re constant. 

For instance, in NumPy, you can do
the following.

In [10]:
x = np.ones(shape=(2, 2))
print(x)

[[1. 1.]
 [1. 1.]]


In [11]:
x[0, 0] = 0.0
print(x)

[[0. 1.]
 [1. 1.]]


Try to do the same thing in TensorFlow, and you will get an error: “EagerTensor object
does not support item assignment.”

In [12]:
x = tf.ones(shape=(2, 2))
print(x)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [13]:
# x[0, 0] = 0.0

```log
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-b9cc6021f76a> in <module>()
----> 1 x[0, 0] = 0.0
      2 print(x)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
```

To train a model, we’ll need to update its state, which is a set of tensors. If tensors aren’t assignable, how do we do it?

That’s where variables come in. `tf.Variable` is the
class meant to manage modifiable state in TensorFlow.

To create a variable, you need to provide some initial value, such as a random tensor.

In [14]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 1.7472106 ],
       [ 1.3933295 ],
       [-0.04593423]], dtype=float32)>


The state of a variable can be modified via its assign method, as follows.

In [15]:
v.assign(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>

It also works for a subset of the coefficients.

In [16]:
v[0, 0].assign((3.0))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

In [17]:
v[1, 0].assign((4.0))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [4.],
       [1.]], dtype=float32)>

Similarly, assign_add() and assign_sub() are efficient equivalents of += and -=, as shown next.

In [18]:
v.assign_add(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [5.],
       [2.]], dtype=float32)>

In [19]:
v.assign_sub(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [4.],
       [1.]], dtype=float32)>

##Tensor operations: Doing math in TensorFlow

Just like NumPy, TensorFlow offers a large collection of tensor operations to express
mathematical formulas.

In [20]:
a = tf.ones((2, 2))
print(a)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [21]:
b = tf.square(a)  # Take the square
print(b)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [22]:
c = tf.sqrt(a)  # Take the square root
print(c)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [23]:
d = b + c  # Add two tensors (element-wise).
print(d)

tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)


In [24]:
e = tf.matmul(a, b)  # Take the product of two tensors
print(e)

tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)


In [25]:
e *= d   # Multiply two tensors
print(e)

tf.Tensor(
[[4. 4.]
 [4. 4.]], shape=(2, 2), dtype=float32)


Importantly, each of the preceding operations gets executed on the fly: at any point,
you can print what the current result is, just like in NumPy. We call this eager execution.

##A look at the GradientTape API

So far, TensorFlow seems to look a lot like NumPy. But here’s something NumPy can’t do: retrieve the gradient of any differentiable expression with respect to any of its inputs. 

Just open a GradientTape scope, apply some computation to one or several input tensors, and retrieve the gradient of the result with respect to the inputs.

In [26]:
input_var = tf.Variable(initial_value=3.)

with tf.GradientTape() as tape:
  result = tf.square(input_var)

gradient = tape.gradient(result, input_var)
print(gradient)

tf.Tensor(6.0, shape=(), dtype=float32)


This is most commonly used to retrieve the gradients of the loss of a model with
respect to its weights: `gradients = tape.gradient(loss, weights)`.

So far, you’ve only seen the case where the input tensors in `tape.gradient()` were
TensorFlow variables. 

It’s actually possible for these inputs to be any arbitrary tensor.
However, only trainable variables are tracked by default. With a constant tensor, you’d
have to manually mark it as being tracked by calling `tape.watch()` on it.

In [27]:
input_const = tf.constant(3.)

with tf.GradientTape() as tape:
  tape.watch(input_const)
  result = tf.square(input_const)

gradient = tape.gradient(result, input_const)
print(gradient)

tf.Tensor(6.0, shape=(), dtype=float32)


To avoid wasting resources, the tape needs to know what to watch. Trainable variables
are watched by default because computing the gradient of a loss with regard to a list of
trainable variables is the most common use of the gradient tape.

The gradient tape is a powerful utility, even capable of computing second-order gradients,
that is to say, the gradient of a gradient. 

For instance, the gradient of the position
of an object with regard to time is the speed of that object, and the second-order
gradient is its acceleration.

If you measure the position of a falling apple along a vertical axis over time and
find that it verifies `position(time) = 4.9 * time ** 2`, what is its acceleration? 

Let’s use two nested gradient tapes to find out.

In [28]:
time = tf.Variable(0.0)

with tf.GradientTape() as outer_tape:
  with tf.GradientTape() as inner_tape:
    position = 4.9 * time ** 2
  speed = inner_tape.gradient(position, time)

# We use the outer tape to compute the gradient of the gradient from the inner tape. Naturally, the answer is 4.9 * 2 = 9.8.
acceleration = outer_tape.gradient(speed, time)
print(speed)
print(acceleration)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(9.8, shape=(), dtype=float32)


##An end-to-end example: A linear classifier in pure TensorFlow